# Lab 6 Summarization (Aknur Shailyatova)

Task: Automatically build abstracts of text documents.
Input: An array of texts in JSON format. Sample texts are at example_texts.json.
Output: An array of abstracts in JSON format (the order of abstracts corresponds to the order of texts in the input data).

The maximum size of each of the abstracts is 300 characters (including white space). If the size of the abstract exceeds the specified threshold, then only the first 300 characters will be evaluated. A trivial solution (the first 300 characters of the document) is allowed, but not welcome.

You can briefly describe the solution in the first line of the download file after the # character. The information will be useful to the authors of the course to gain an idea of the methods and approaches used.

Rating: ROUGE-2 - proximity to a set of manually compiled abstracts based on bigrams of words (value from 0 to 1).
Input can be found at dataset_43428_1.txt
Output is at outdataset_43428_1.txt

In solution was used Book "Text Analytics with Python", Dipanjan Sarkar, Chapter 6.  


In [1]:
import pandas as pd
import numpy as np
import re
from gensim.summarization import summarize
from nltk import tokenize
from rouge import Rouge 


df = pd.read_json('dataset_43428_1.txt')
df.head()

,0
0,В Сербии арестован последний военный преступни...
1,АнтиПоттер\nРуководство по эксплуатации волшеб...
2,Авто по карману\nМихаил Прохоров собирается за...
3,В Казахстане состоялись выборы Президента\n\nВ...
4,В Москве подвели итоги Международной конференц...


Testing methods using the book mentioned above.

In [2]:
sample = 'арбофос\n\nКарбофос ( O , O - Диметил - S - ( 1,2 - дикарбэтоксиэтил ) дитиофосфат , малатион ) — фосфорорганическое соединение , инсектицид широкого спектра действия , акарицид .\n\nПрименение\n\nПрименялся против комаров , мух , клещей , клопов , паразитов , повреждающих фруктовые деревья , овощные и декоративные растения .\nВ настоящее время вытеснен более совершенными и безопасными для человека пиретроидами .\nПрепараты на основе карбофоса остаются одними из самых эффективных средств борьбы против постельных клопов , весьма слабо чувствительных к прочим инсектицидам .\n\nМеханизм действия\n\nАнтихолинэстеразное средство необратимого действия .\nНа длительное время инактивирует ацетилхолинэстеразу ( фермент , гидролизующий ацетилхолин в холинэргических синапсах ) , в результате чего происходит изменение в структуре фермента , и его инактивация становится необратимой .\nЗначительно усиливает и удлиняет действие ацетилхолина .'
print(sample)
word_list = tokenize.word_tokenize(sample)
print(len(word_list))
print()
sample = re.sub(r'\n|\r', ' ', sample)
sample = re.sub(r' +', ' ', sample)
sample = sample.strip()
sample1 = summarize(sample, ratio=0.2, split=False)
print(sample1)
word_list = tokenize.word_tokenize(sample1)
print(len(word_list))
print()
sample2 = summarize(sample, word_count=75, split=False)
print(sample2)
word_list = tokenize.word_tokenize(sample2)
print(len(word_list))
print()

арбофос

Карбофос ( O , O - Диметил - S - ( 1,2 - дикарбэтоксиэтил ) дитиофосфат , малатион ) — фосфорорганическое соединение , инсектицид широкого спектра действия , акарицид .

Применение

Применялся против комаров , мух , клещей , клопов , паразитов , повреждающих фруктовые деревья , овощные и декоративные растения .
В настоящее время вытеснен более совершенными и безопасными для человека пиретроидами .
Препараты на основе карбофоса остаются одними из самых эффективных средств борьбы против постельных клопов , весьма слабо чувствительных к прочим инсектицидам .

Механизм действия

Антихолинэстеразное средство необратимого действия .
На длительное время инактивирует ацетилхолинэстеразу ( фермент , гидролизующий ацетилхолин в холинэргических синапсах ) , в результате чего происходит изменение в структуре фермента , и его инактивация становится необратимой .
Значительно усиливает и удлиняет действие ацетилхолина .
131

Препараты на основе карбофоса остаются одними из самых эффективных 

Making summarization according the book.

In [9]:
#Model

def OriginalTextLength(field):
    text = field[0]
    word_tokens = tokenize.word_tokenize(text)
    return len(word_tokens)

def TextCleaning(field):
    text = field[0]
    text = text.lower()
    text = re.sub(r'\n|\r', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()
    return text

def TextSummary(field):
    text = field['Cleaned Text']
    sentence_tokens = tokenize.sent_tokenize(text)
    word_tokens = tokenize.word_tokenize(text)
    initial_word_count = len(word_tokens)
    if (len(sentence_tokens) == 1):
        return text
    if (initial_word_count <= 1000):
        text_summary = summarize(text, ratio = 0.1, split = False)
    elif (initial_word_count >= 1000):
        text_summary = summarize(text, word_count = 300, split = False)
    
    return text_summary

def SummarizedTextLength(field):
    text = field['Summarization']
    word_tokens = tokenize.word_tokenize(text)
    return len(word_tokens)

df['Initial Text Length'] = df.apply(lambda field: OriginalTextLength(field), axis=1)
df['Cleaned Text'] = df.apply(lambda field: TextCleaning(field), axis=1)
df['Summarization'] = df.apply(lambda field: TextSummary(field), axis=1)
df['Summarization Length'] = df.apply(lambda field: SummarizedTextLength(field), axis=1)

#Metric

# def RougeMetric(field):
#     summa = field['Summarization']
#     summaryH = field['']
#     rouge = Rouge()
#     estimates = rouge.get_scores(hyposummathesis, summaryH)
#     return estimates[0]['ROUGE-2']

# for st in df['Summarization']:
#     print('Summarization:',st,'\n')
    
# df['ROUGE-2'] = df.apply(lambda field: RougeMetric(field), axis=1)
# df

df.to_json('outdataset_43428_1.txt')
df.head()

,0,Initial Text Length,Cleaned Text,Summarization,Summarization Length
0,В Сербии арестован последний военный преступни...,213,в сербии арестован последний военный преступни...,"за информацию , способную помочь в поимке гора...",20
1,АнтиПоттер\nРуководство по эксплуатации волшеб...,1918,антипоттер руководство по эксплуатации волшебн...,мне уже говорили мои молодые коллеги и просто ...,296
2,Авто по карману\nМихаил Прохоров собирается за...,110,авто по карману михаил прохоров собирается зан...,,0
3,В Казахстане состоялись выборы Президента\n\nВ...,376,в казахстане состоялись выборы президента в во...,по предварительным данным опроса избирателей н...,33
4,В Москве подвели итоги Международной конференц...,880,в москве подвели итоги международной конференц...,в москве подвели итоги международной конференц...,188


In [10]:
outdf = pd.read_json('outdataset_43428_1.txt')
outdf.head()

,0,Initial Text Length,Cleaned Text,Summarization,Summarization Length
0,В Сербии арестован последний военный преступни...,213,в сербии арестован последний военный преступни...,"за информацию , способную помочь в поимке гора...",20
1,АнтиПоттер\nРуководство по эксплуатации волшеб...,1918,антипоттер руководство по эксплуатации волшебн...,мне уже говорили мои молодые коллеги и просто ...,296
10,Кризис массового обслуживания\n\nЕсть отрасли ...,216,"кризис массового обслуживания есть отрасли , в...","кризис массового обслуживания есть отрасли , в...",25
100,"Статистическое бюро сообщает , что средняя зар...",374,"статистическое бюро сообщает , что средняя зар...","статистическое бюро сообщает , что средняя зар...",61
101,2.5.2 .\nИскусство « разговора » в современном...,680,2.5.2 . искусство « разговора » в современном ...,искусство « разговора » в современном искусств...,198


In [11]:
outdf.to_csv('outdataset_43428_1.csv', header=False, index=False)